<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-13 13:44:56
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.46 C
-------------------
Today PnL: 12.48 K (0.09%)
Current PnL: -19.54 L (-12.8%)
CY Booked + Current PnL: -6.02 L (-3.94%)
-------------------
Total profit:  2.12 L
Total loss:  -21.65 L
-------------------
Total Booked + Current PnL: 20.96 L (16.69%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.27%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.35 C
Est FTT PnL: 88.84 L (60.9%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.7%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,102.35,52.0,M-SC,2.71,87167.0,-13610.0,13703.0,0.21,-13.51,15.72,0.09,245.0,-0.99,0.60,14.52,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.00,141.07,83.0,M-SC,3.19,158212.0,-16347.0,16375.0,3.52,-9.36,10.35,0.02,221.0,-1.00,1.10,37.62,OX40N,NTT,PAINTS
2,ABBOTINDIA,35195.00,-10.45,45.0,X-MC,3.36,88380.0,-2298.0,17208.0,-0.37,-2.53,19.47,16.44,101.0,-0.13,0.61,17.13,X40,ATH,PHARMA
34,ICICIGI,2252.93,-11.98,61.0,X-MC,7.55,190129.0,15809.0,19393.0,0.78,9.07,10.20,20.19,91.0,0.82,1.32,25.55,X40,ATH,INSURANCE
42,ITC,452.00,-38.34,47.0,X-LC,0.88,199170.0,-968.0,21849.0,0.07,-0.48,10.97,10.44,4.0,-0.04,1.38,5.05,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.54,86.0,X-LC,7.98,253686.0,1918.0,51295.0,4.08,0.76,20.22,21.13,27.0,0.04,1.76,36.46,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,141.07,83.0,M-SC,3.19,158212.0,-16347.0,16375.0,3.52,-9.36,10.35,0.02,221.0,-1.00,1.10,37.62,OX40N,NTT,PAINTS
13,BERGEPAINT,680.00,-13.11,70.0,X-MC,1.62,233726.0,6387.0,41673.0,3.04,2.81,17.83,21.14,106.0,0.15,1.62,31.74,XY24,NTT,PAINTS
5,ANGELONE,3033.00,27.04,74.0,X-SC,7.11,224639.0,33633.0,24059.0,2.24,17.61,10.71,30.21,157.0,1.40,1.56,42.17,X40N,NTT,FINANCE
64,SHALBY,327.00,1086.60,43.0,M-SC,16.47,161330.0,-19813.0,65274.0,1.75,-10.94,40.46,25.10,235.0,-0.30,1.12,29.29,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-54.55,54.0,H-SC,9.51,259714.0,-26791.0,284127.0,-4.39,-9.35,109.40,89.82,134.0,-0.09,1.80,37.41,XY24,NTT,FINANCE
15,CAMPUS,393.00,-26.26,40.0,M-SC,4.08,153573.0,-14202.0,70045.0,-2.07,-8.47,45.61,33.28,210.0,-0.20,1.06,19.03,XY24,NTT,FOOTWEAR
67,SONACOMS,806.63,-31.27,62.0,M-SC,6.37,85461.0,-15698.0,55695.0,-1.89,-15.52,65.17,39.54,202.0,-0.28,0.59,20.51,AR,ATH,AUTO
7,ASIANTILES,137.00,7544.44,64.0,L-SC,15.82,83846.0,-9964.0,86579.0,-1.82,-10.62,103.26,81.67,269.0,-0.12,0.58,62.37,XR,NTT,CERAMICS
69,SURYODAY,240.00,51.32,40.0,H-SC,13.55,139128.0,-39943.0,105668.0,-1.54,-22.31,75.95,36.71,135.0,-0.38,0.96,38.04,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,ZYDUSLIFE,1286.17,-9.48,37.0,H-MC,5.89,208845.0,-729.0,75393.0,-0.67,-0.35,36.1,35.63,NaN,-0.01,1.45,16.27,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.0,141.07,83.0,M-SC,3.19,158212.0,-16347.0,16375.0,3.52,-9.36,10.35,0.02,221.0,-1.00,1.10,37.62,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.0,102.35,52.0,M-SC,2.71,87167.0,-13610.0,13703.0,0.21,-13.51,15.72,0.09,245.0,-0.99,0.60,14.52,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.42,52.0,H-SC,2.40,224316.0,-45351.0,81673.0,1.51,-16.82,36.41,13.47,138.0,-0.56,1.55,14.06,XY24,NTT,PAINTS
66,SIS,528.0,2029.53,49.0,H-SC,2.41,85935.0,-25097.0,48708.0,0.03,-22.60,56.68,21.26,156.0,-0.52,0.60,16.11,OX40N,NTT,MISC
72,TATAELXSI,9161.0,-24.89,47.0,H-MC,6.30,101688.0,-30147.0,72371.0,-0.67,-22.87,71.17,32.03,98.0,-0.42,0.70,13.90,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,56.0,M-LC,5.44,152508.0,1563.0,107899.0,0.31,1.04,70.75,72.51,53.0,0.01,1.06,7.12,XR,NTT,IT
37,INDIAMART,4810.62,-52.77,58.0,H-SC,4.77,131832.0,8496.0,123131.0,-0.68,6.89,93.40,106.72,119.0,0.07,0.91,31.88,AR,ATH,MISC
61,SAMMAANCAP,326.00,-193.18,56.0,M-SC,36.10,160407.0,11187.0,132993.0,0.71,7.50,82.91,96.62,208.0,0.08,1.11,73.60,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.77,58.0,H-SC,4.77,131832.0,8496.0,123131.0,-0.68,6.89,93.40,106.72,119.0,0.07,0.91,31.88,AR,ATH,MISC
84,WIPRO,420.00,-13.34,56.0,M-LC,5.44,152508.0,1563.0,107899.0,0.31,1.04,70.75,72.51,53.0,0.01,1.06,7.12,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.48,37.0,H-MC,5.89,208845.0,-729.0,75393.0,-0.67,-0.35,36.10,35.63,NaN,-0.01,1.45,16.27,AR,ATH,PHARMA
83,WHIRLPOOL,2270.00,-45.97,35.0,M-SC,9.14,112019.0,-2779.0,96818.0,-0.76,-2.42,86.43,81.92,223.0,-0.03,0.78,32.61,XR,NTT,DURABLES
47,KPIGREEN,731.05,1.34,41.0,H-SC,10.25,120859.0,-4438.0,63366.0,-1.30,-3.54,52.43,47.03,141.0,-0.07,0.84,51.33,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-18.14,27.0,X-MC,5.79,251655.0,-2241.0,84556.0,-1.42,-0.88,33.60,32.42,105.0,-0.03,1.74,14.08,X40,ATH,FMCG
1,ABB,7934.00,-43.56,29.0,H-LC,5.17,235968.0,-25651.0,144861.0,-0.16,-9.80,61.39,45.57,7.0,-0.18,1.63,0.41,AR,NTT,ELECTRICAL
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87158.0,-41512.0,86809.0,-0.70,-32.26,99.60,35.20,219.0,-0.48,0.60,0.31,X40,NTT,FOOTWEAR
54,QUESS,986.00,-52.07,30.0,X-SC,37.26,47370.0,-17636.0,168566.0,-1.41,-27.13,355.85,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
45,JSWINFRA,342.00,-24.57,31.0,X-MC,5.04,189405.0,-10942.0,41442.0,-1.42,-5.46,21.88,15.23,178.0,-0.26,1.31,25.03,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.34,47.0,X-LC,0.88,199170.0,-968.0,21849.0,0.07,-0.48,10.97,10.44,4.0,-0.04,1.38,5.05,X40,NTT,FMCG
53,PGHH,17907.65,-32.93,32.0,X-MC,1.55,194100.0,-6720.0,74515.0,0.54,-3.35,38.39,33.76,80.0,-0.09,1.34,1.68,X40,ATH,FMCG
65,SIEMENS,4671.50,-4.11,46.0,H-LC,1.65,153980.0,-32115.0,79592.0,0.75,-17.26,51.69,25.51,15.0,-0.40,1.07,13.47,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-7.45,46.0,X-MC,3.20,308868.0,-19123.0,125648.0,0.88,-5.83,40.68,32.48,92.0,-0.15,2.14,4.93,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-6.09,62.0,X-LC,3.28,246218.0,12074.0,36539.0,-0.29,5.16,14.84,20.77,86.0,0.33,1.71,12.43,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.07,30.0,X-SC,37.26,47370.0,-17636.0,168566.0,-1.41,-27.13,355.85,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87158.0,-41512.0,86809.0,-0.70,-32.26,99.60,35.20,219.0,-0.48,0.60,0.31,X40,NTT,FOOTWEAR
1,ABB,7934.00,-43.56,29.0,H-LC,5.17,235968.0,-25651.0,144861.0,-0.16,-9.80,61.39,45.57,7.0,-0.18,1.63,0.41,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.93,32.0,X-MC,1.55,194100.0,-6720.0,74515.0,0.54,-3.35,38.39,33.76,80.0,-0.09,1.34,1.68,X40,ATH,FMCG
10,BAJAJHFL,181.50,-15.96,38.0,X-MC,6.42,176940.0,-23566.0,120726.0,-0.13,-11.75,68.23,48.45,90.0,-0.20,1.23,2.03,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.07,30.0,X-SC,37.26,47370.0,-17636.0,168566.0,-1.41,-27.13,355.85,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.75,42.0,X-SC,3.58,80812.0,-64348.0,140273.0,-1.09,-44.33,173.58,52.31,136.0,-0.46,0.56,8.17,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87158.0,-41512.0,86809.0,-0.70,-32.26,99.60,35.20,219.0,-0.48,0.60,0.31,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,45.0,X-MC,3.36,88380.0,-2298.0,17208.0,-0.37,-2.53,19.47,16.44,101.0,-0.13,0.61,17.13,X40,ATH,PHARMA
33,HONAUT,58357.33,-24.36,37.0,X-SC,0.97,106650.0,-21288.0,68427.0,0.15,-16.64,64.16,36.84,143.0,-0.31,0.74,9.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.44,62.0,X-LC,8.77,293637.0,-52319.0,119011.0,-0.26,-15.12,40.53,19.28,1.0,-0.44,2.03,8.55,X40,ATH,IT
40,INFY,2275.00,-14.93,68.0,X-LC,4.31,331322.0,18434.0,153270.0,0.25,5.89,46.26,54.87,3.0,0.12,2.29,14.66,X40,BTT,IT
42,ITC,452.00,-38.34,47.0,X-LC,0.88,199170.0,-968.0,21849.0,0.07,-0.48,10.97,10.44,4.0,-0.04,1.38,5.05,X40,NTT,FMCG
81,VBL,671.64,-18.84,46.0,X-LC,5.86,292300.0,-23542.0,136212.0,-0.25,-7.45,46.60,35.67,5.0,-0.17,2.02,5.37,X40N,ATH,FMCG
1,ABB,7934.00,-43.56,29.0,H-LC,5.17,235968.0,-25651.0,144861.0,-0.16,-9.80,61.39,45.57,7.0,-0.18,1.63,0.41,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.00,304.06,79.0,L-MC,6.70,49621.0,-27586.0,52980.0,0.71,-35.73,106.77,32.89,258.0,-0.52,0.34,36.65,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,46.0,L-SC,22.25,77875.0,-35674.0,75726.0,0.27,-31.42,97.24,35.27,268.0,-0.47,0.54,90.60,XR,NTT,HOTELS
7,ASIANTILES,137.00,7544.44,64.0,L-SC,15.82,83846.0,-9964.0,86579.0,-1.82,-10.62,103.26,81.67,269.0,-0.12,0.58,62.37,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.15,50.0,H-SC,7.52,92055.0,-5925.0,27524.0,-0.37,-6.05,29.90,22.05,152.0,-0.22,0.64,33.81,XR,ATH,FINANCE
37,INDIAMART,4810.62,-52.77,58.0,H-SC,4.77,131832.0,8496.0,123131.0,-0.68,6.89,93.40,106.72,119.0,0.07,0.91,31.88,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.54,86.0,X-LC,7.98,253686.0,1918.0,51295.0,4.08,0.76,20.22,21.13,27.0,0.04,1.76,36.46,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,141.07,83.0,M-SC,3.19,158212.0,-16347.0,16375.0,3.52,-9.36,10.35,0.02,221.0,-1.00,1.10,37.62,OX40N,NTT,PAINTS
79,VAIBHAVGBL,521.00,73.55,62.0,H-SC,7.58,150475.0,-32300.0,143373.0,0.72,-17.67,95.28,60.77,125.0,-0.23,1.04,35.51,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7544.44,64.0,L-SC,15.82,83846.0,-9964.0,86579.0,-1.82,-10.62,103.26,81.67,269.0,-0.12,0.58,62.37,XR,NTT,CERAMICS
5,ANGELONE,3033.00,27.04,74.0,X-SC,7.11,224639.0,33633.0,24059.0,2.24,17.61,10.71,30.21,157.0,1.40,1.56,42.17,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.78
1,25,44.84
2,50,76.41


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.24
LC    27.92
MC    26.98
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.69
X40      20.78
X40N     12.35
XY25     10.12
XR        9.69
AR        9.06
OX40N     6.56
X200      1.84
SR        1.03
MH        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.71
X-MC    22.88
X-LC    19.95
M-SC    12.48
X-SC     5.60
H-LC     4.54
H-MC     3.76
M-LC     2.44
L-SC     1.45
L-LC     0.99
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.72,-4.14,37.81
IT,12.74,-14.19,75.31
FINANCE,12.00,-9.15,58.62
MISC,7.65,-17.52,76.80
PAINTS,6.03,-6.14,21.96
ELECTRICAL,5.58,-12.19,51.96
INSURANCE,4.50,2.53,31.82
AUTO,3.53,-12.75,120.84
PHARMA,3.46,-0.14,33.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3307771.0,20
AR,1263397.0,10
XR,1256640.0,13
X40,967159.0,14
X40N,686432.0,9
OX40N,536970.0,9
XY25,472589.0,7
SR,270619.0,2
MH,63366.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3403382.0,24
X-MC,1382614.0,16
M-SC,1243729.0,15
X-LC,769921.0,11
X-SC,541167.0,6
H-MC,329680.0,3
H-LC,283135.0,3
L-SC,251398.0,3
M-LC,153882.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1179570.0      6
           AR         839150.0      5
           XR         788939.0      7
M-SC       XY24       728050.0      6
X-MC       XY24       550793.0      4
           X40        485095.0      8
X-LC       X40        395255.0      5
X-SC       X40N       285792.0      4
H-SC       SR         270619.0      2
           OX40N      261738.0      3
X-LC       X40N       238472.0      3
H-LC       AR         224453.0      2
X-MC       XY25       184558.0      2
H-MC       XY24       181916.0      1
X-SC       XY24       168566.0      1
L-SC       XR         162305.0      2
X-MC       X40N       162168.0      2
M-SC       XR         144517.0      2
           XY25       132993.0      1
           AR         124401.0      2
           OX40N      113768.0      4
M-LC       XR         107899.0      1
L-SC       OX40N       89093.0      1
X-SC       X40         86809.0      1
H-MC       AR          75393.0      1
           OX40N       72371.0      1
X-LC       XY25        72298.0      2
           XY24        63896.0      1
H-SC       MH          63366.0      1
H-LC       X200        58682.0      1
L-MC       XR          52980.0      1
M-LC       XY25        45983.0      1
L-LC       XY25        36757.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
